In [1]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts.xml")
!pip3 install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#creating index
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
inverted_index = {}
def update_index(text, document_id):
  #formats text
  temp_tokens = word_tokenize(re.sub(r'[^a-z]', ' ', text))
  tokens = [i for i in temp_tokens if i not in stop_words]

  #runs through all tokens and indexed accordingly
  for token in tokens:
    #if token not in stopwords.words('english'):
      if token in inverted_index:
        if document_id in inverted_index[token]:
          inverted_index[token][document_id] += 1
        else:
          inverted_index[token][document_id] = 1
      else:
        inverted_index[token] = {document_id: 1}

#indexes questions
print("indexing questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  title = question.title.lower().strip()
  body = question.body.lower().strip()
  text = title + " " + body
  update_index(text, question_id)
print("complete")

#indexes answers
print("indexing answers...")
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  text = answer.body.lower().strip()
  update_index(text, answer_id)
print("Complete")
#print(inverted_index)


indexing questions...
complete
indexing answers...
Complete


In [4]:
#run to simply initalize this function for use 
from re import A

#searches index for document matches
def search(keys,query,query_num):
  temp_dic = {}
  missed_keys = []
  
  #goes through each word in query
  for word in query:   
    for current_key in keys:
      if current_key in inverted_index[word]:
        if current_key not in missed_keys:
          if current_key in temp_dic:
            temp_dic[current_key] += inverted_index[word][current_key]
          else:
            temp_dic[current_key] = inverted_index[word][current_key]
      else:
        missed_keys.append(current_key)
        if current_key in temp_dic:
          temp_dic.pop(current_key)

  #reverse sorts, by largest amount of occurrence first
  myList = sorted(temp_dic.items(), key=lambda x: x[1], reverse=True)
  i = 0
  for items in myList:
    #makes sure only 10 of each pass are printed
    if i <= 49:
      print(str(items[0])+"    "+str(items[1]))
      i += 1
    else:
      break
################################################################################
#Querys
#simply add Querys to list 
querys = ["weathering effects", "fix shading problem","Nurbs path doesn't go fully closed","Problem with bone","Scaling faces with drivers",
"Glowing text effect","Creating a hole out of an custom shape","Brick texture with multiple colors","How to Dribble soccer ball","Masking selected vertices for sculpting",
"How to use Modular Trees","Netrender addon missing","drawing a belt around a gear","Constrain vertex positions to moving objects","Deformation of particle objects",
"change smoke color","Mix shader is not working","Scale deformation when rotating imported FBX","cubic trefoil knot","How to get the absolute path"]

keys = []
query_num = 0

#scans through and prints each query
for query in querys:
  print("Searching: "+query)
  #seperates query into a list
  seperated_query = [i for i in query.lower().split(" ") if i not in stop_words]
  #scans through each word in query specifically to save all matching occurances to the list of keys
  for query_part in seperated_query:
    if query_part in inverted_index:
      #adds keys of matching documents for current word in query
      tempkeys = list(inverted_index[query_part].keys() )
  #sorts keys to keep organization for my sanity
  keys.sort()
  ordered_keys = []
  #removes duplicate keys
  keys = []
  for key in tempkeys:
    if key not in keys:
      keys.append(key)

  #uses function to search index and print final result
  search(keys,seperated_query,query_num)
  query_num += 1
  keys = []
  

Searching: weathering effects
69306    6
55991    4
146393    4
163723    2
166557    2
242783    2
Searching: fix shading problem
6296    15
61442    11
208955    11
62366    9
147663    9
207012    9
227483    9
52354    9
77864    8
110642    8
253641    8
75106    8
196035    8
35759    7
106700    7
182149    7
182282    7
216364    7
228618    7
234873    7
239076    7
71653    6
77376    6
82863    6
116532    6
128401    6
168621    6
177822    6
193414    6
223845    6
230667    6
244573    6
246104    6
256738    6
128465    6
170771    6
186821    6
228394    6
42221    5
65717    5
70688    5
86290    5
88435    5
148717    5
149330    5
157970    5
170414    5
172926    5
181512    5
184963    5
Searching: Nurbs path doesn't go fully closed
146128    7
Searching: Problem with bone
63056    37
245063    30
244712    28
111192    26
180572    26
233640    24
116128    24
126641    23
220377    22
263661    22
165195    21
248340    21
48489    21
9143    20
39924    20
63878

Boolean Retrieval